In [11]:
!pip install beautifulsoup4

In [13]:
import re
from bs4 import BeautifulSoup
import pandas as pd


In [15]:
with open("female-detainee-cases.html", "r", encoding="utf-8") as f:
    html = f.read()
soup = BeautifulSoup(html, "html.parser")


In [21]:
anchors = soup.find_all("a", href=True)
# Keep only hrefs like "1234.html"
case_links = [a for a in anchors if re.fullmatch(r"\d+\.html", a["href"])]


In [23]:
records = []
pattern = re.compile(r"^Case\s+(\d+)\s+(.*)$")
for a in case_links:
    href_num = int(a["href"].split(".")[0])
    text = a.get_text(strip=True)
    m = pattern.match(text)
    if m:
        text_num = int(m.group(1))
        name = m.group(2)
    else:
        # unexpected format
        text_num = None
        name = text
    records.append({
        "href": a["href"],
        "case_id_href": href_num,
        "case_id_text": text_num,
        "name": name,
        "flag_mismatch": (text_num is not None and text_num != href_num)
    })


In [27]:
df = pd.DataFrame(records)
# Normalize whitespace in names
df["name"] = df["name"].str.replace(r"\s+", " ", regex=True).str.strip()
# Drop intermediate columns if you like, or keep them for audit


In [29]:
df.to_csv("female_detainee_cases.csv", index=False)
